<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/Fine_tuning_Transformers_for_typo_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Small BERT for typo detection using NER

> Author: [Manuel Romero/mrm8488](https://twitter.com/mrm8488)

## Download the dataset

In [1]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz

--2020-04-25 14:03:35--  https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
Resolving github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)... 52.216.99.179
Connecting to github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)|52.216.99.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43769081 (42M) [application/x-gzip]
Saving to: ‘github-typo-corpus.v1.0.0.jsonl.gz’

github-typo-corpus. 100%[===================>]  41.74M  33.0MB/s    in 1.3s    

2020-04-25 14:03:36 (33.0 MB/s) - ‘github-typo-corpus.v1.0.0.jsonl.gz’ saved [43769081/43769081]



In [0]:
!gzip -d /content/github-typo-corpus.v1.0.0.jsonl.gz

## Inspect the dataset

In [11]:
import json
i=0

with open('/content/github-typo-corpus.v1.0.0.jsonl', 'r') as f:
  sample_file = f.readlines()
  for sample in sample_file:
    obj = json.loads(sample)
    print(obj['edits'][0]['src']['lang'])
    print(obj['edits'][0]['src']['text'])
    print(obj['edits'][0]['tgt']['text'])
    print(obj)
    i=i+1
    if(i > 20):
      break
              

eng
            IN.User.authorize(function(){
            IN.User.authorize(function() {
{'repo': 'https://github.com/abacritt/angularx-social-login', 'commit': 'd4c912f5ccd70c81f424fadbf1fe1a2ecb942f07', 'message': 'Fix typo\n', 'edits': [{'src': {'text': '            IN.User.authorize(function(){', 'path': 'src/lib/src/providers/linkedIn-login-provider.ts', 'lang': 'eng', 'ppl': 14.759690820409018}, 'tgt': {'text': '            IN.User.authorize(function() {', 'path': 'src/lib/src/providers/linkedIn-login-provider.ts', 'lang': 'eng', 'ppl': 13.02657495778316}, 'prob_typo': 0.9393920322691369, 'is_typo': True}, {'src': {'text': '            IN.User.logout(function(){', 'path': 'src/lib/src/providers/linkedIn-login-provider.ts', 'lang': 'eng', 'ppl': 21.30207029147685}, 'tgt': {'text': '            IN.User.logout(function() {', 'path': 'src/lib/src/providers/linkedIn-login-provider.ts', 'lang': 'eng', 'ppl': 18.466537526108358}, 'prob_typo': 0.9433146599837509, 'is_typo': True}]}
eng
 

In [19]:
import json
j=0

with open('/content/github-typo-corpus.v1.0.0.jsonl', 'r') as f:
  sample_file = f.readlines()
  for sample in sample_file:
    obj = json.loads(sample)
    if(len(obj['edits'][0]['tgt']['text'].split()) == 1 and obj['edits'][0]['src']['lang'] == "eng"):
      if(j < 20):
        print(obj['edits'][0]['tgt']['text'].strip())
      j=j+1

print(j)

:nocomments:
:excerpts:
Finally,
#Staging
if(!this.preReady){
"Libraries/unrar/dll.cpp"
@updateMessage()
groupPaletteColors:
"index.html"
**Requirements:**
Component={AppPage}
alert('error')
http://amplitudejs.com
e.target.blur();
"SysPermissionId"),
console.error(err.printout);
super(app);
util.Error(err.Error())
.x_label_area_size(30)
add_definitions(-std=c++14)
8515


## Create a new dataset with 'NER' style

In [0]:
import json


with open('/content/github-typo-corpus.v1.0.0.jsonl', 'r') as f:
  sample_file = f.readlines()
  for sample in sample_file:
    obj = json.loads(sample)
    if(len(obj['edits'][0]['tgt']['text'].split()) == 1 and obj['edits'][0]['src']['lang'] == "eng"):
      with open('train.txt', 'a') as f2:
         print(obj['edits'][0]['src']['text'].strip() + " " + "typo", file=f2)
         print(obj['edits'][0]['tgt']['text'].strip() + " " + "ok", file=f2)

In [33]:
!head -20 train.txt

:nocoments: typo
:nocomments: ok
:excerpt: typo
:excerpts: ok
Finnally, typo
Finally, ok
#Stagging typo
#Staging ok
if(this.preReady){ typo
if(!this.preReady){ ok
"Libraries/unrar/dll.cpp", typo
"Libraries/unrar/dll.cpp" ok
@udpateMessage() typo
@updateMessage() ok
gorupPaletteColors: typo
groupPaletteColors: ok
"index.html", typo
"index.html" ok
Requirements typo
**Requirements:** ok


In [34]:
!wc -l train.txt

17030 train.txt


In [0]:
!cat train.txt | tail -2500 > dev.txt
!cat train.txt | head -14530 > train2.txt
!rm train.txt
!mv train2.txt train.txt

## Install required dependecies

In [5]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 24693 (delta 16), reused 25 (delta 7), pack-reused 24646
Receiving objects: 100% (24693/24693), 14.92 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (17348/17348), done.


In [6]:
!pip install -q ./transformers
!pip install -q tensorboardX
!pip install -q seqeval

     |████████████████████████████████| 3.8MB 4.6MB/s 
     |████████████████████████████████| 1.0MB 51.2MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
     |████████████████████████████████| 204kB 4.9MB/s 


## Preprocess the new dataset

In [0]:
!mkdir ner_dataset

In [8]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2020-04-25 14:04:13--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 923 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     923  --.-KB/s    in 0s      

2020-04-25 14:04:13 (167 MB/s) - ‘preprocess.py’ saved [923/923]



In [9]:
!python3 preprocess.py train.txt "google/bert_uncased_L-4_H-512_A-8" 128 > /content/ner_dataset/train.txt
!python3 preprocess.py dev.txt "google/bert_uncased_L-4_H-512_A-8" 128 > /content/ner_dataset/dev.txt

2020-04-25 14:04:18.393982: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 383/383 [00:00<00:00, 402kB/s]
Downloading: 100% 232k/232k [00:00<00:00, 911kB/s]
2020-04-25 14:04:26.541762: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [0]:
!rm train.txt
!rm dev.txt

## Create the labels

In [0]:
!touch labels.txt

In [0]:
!echo typo >> labels.txt
!echo -n ok >> labels.txt

## Fine-tune the model [BERT-Small](https://huggingface.co/google/bert_uncased_L-4_H-512_A-8) for NER task

In [12]:
!python /content/transformers/examples/ner/run_ner.py --data_dir /content/ner_dataset \
--model_type bert \
--labels /content/labels.txt \
--model_name_or_path google/bert_uncased_L-4_H-256_A-4 \
--output_dir model_output \
--max_seq_length 128 \
--num_train_epochs 50 \
--per_gpu_train_batch_size 128 \
--overwrite_output_dir \
--save_steps 1000 \
--do_train \
--do_eval

2020-04-25 14:04:35.629232: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/25/2020 14:04:37 - INFO - transformers.training_args -   PyTorch: setting up devices
04/25/2020 14:04:37 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/25/2020 14:04:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='model_output', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=128, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=50.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1)
04/25/20

## Test our model using 🤗 Transformers ```pipelines```

In [0]:
from transformers import pipeline
ner_pipeline = pipeline("ner",model="/content/model_output", tokenizer="/content/model_output")

In [14]:
result = ner_pipeline("here there is an error in coment")
result[1:-1]

[{'entity': 'ok', 'score': 0.9021041989326477, 'word': 'here'},
 {'entity': 'ok', 'score': 0.7975626587867737, 'word': 'there'},
 {'entity': 'ok', 'score': 0.8596242070198059, 'word': 'is'},
 {'entity': 'ok', 'score': 0.7071516513824463, 'word': 'an'},
 {'entity': 'ok', 'score': 0.943381130695343, 'word': 'error'},
 {'entity': 'ok', 'score': 0.8047608733177185, 'word': 'in'},
 {'entity': 'ok', 'score': 0.8240702152252197, 'word': 'come'},
 {'entity': 'typo', 'score': 0.5004884004592896, 'word': '##nt'}]

In [17]:
result = ner_pipeline("Adddd validation midelware")
result[1:-1]

[{'entity': 'ok', 'score': 0.7128152847290039, 'word': 'add'},
 {'entity': 'typo', 'score': 0.5388424396514893, 'word': '##dd'},
 {'entity': 'ok', 'score': 0.94792640209198, 'word': 'validation'},
 {'entity': 'typo', 'score': 0.5839331746101379, 'word': 'mid'},
 {'entity': 'ok', 'score': 0.5195121765136719, 'word': '##el'},
 {'entity': 'ok', 'score': 0.7222476601600647, 'word': '##ware'}]

In [19]:
result = ner_pipeline("Uploaf foler with configuration files")
result[1:-1]

[{'entity': 'ok', 'score': 0.6706706285476685, 'word': 'up'},
 {'entity': 'typo', 'score': 0.5158016085624695, 'word': '##lo'},
 {'entity': 'ok', 'score': 0.9170085787773132, 'word': '##af'},
 {'entity': 'ok', 'score': 0.8197095394134521, 'word': 'f'},
 {'entity': 'typo', 'score': 0.9499740600585938, 'word': '##ole'},
 {'entity': 'typo', 'score': 0.9682391285896301, 'word': '##r'},
 {'entity': 'typo', 'score': 0.7309896945953369, 'word': 'with'},
 {'entity': 'ok', 'score': 0.7321141958236694, 'word': 'configuration'},
 {'entity': 'ok', 'score': 0.9064017534255981, 'word': 'files'}]

## Upload to Hugginface 🤗 Transformers

In [0]:
!mv model_output bert-small-finetuned-typo-detection

In [21]:
!transformers-cli login

2020-04-25 14:13:02.885589: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: mrm8488
Password: 
Login successful
Your token: YvztrdEASHDOFinxHemIWlpRqxJWWhjrgeMWrFEeTgmzXpGRaaKYGvnjOiGhsexhvGHAzZgpCwZxlIOXtjNQaXfCUoZAAaEkBgQViOmSIzfJIwOuzPlIQLemSaYFFQah 

Your token has been saved to /root/.huggingface/token


In [22]:
!transformers-cli upload ./bert-small-finetuned-typo-detection

2020-04-25 14:13:45.357451: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
About to upload file /content/bert-small-finetuned-typo-detection/vocab.txt to S3 under filename bert-small-finetuned-typo-detection/vocab.txt and namespace mrm8488
About to upload file /content/bert-small-finetuned-typo-detection/special_tokens_map.json to S3 under filename bert-small-finetuned-typo-detection/special_tokens_map.json and namespace mrm8488
About to upload file /content/bert-small-finetuned-typo-detection/tokenizer_config.json to S3 under filename bert-small-finetuned-typo-detection/tokenizer_config.json and namespace mrm8488
About to upload file /content/bert-small-finetuned-typo-detection/config.json to S3 under filename bert-small-finetuned-typo-detection/config.json and namespace mrm8488
About to upload file /content/bert-small-finetuned-typo-detection/training_args.bin to S3 under filename bert-small-finetuned-typo-detecti

# Fine-tuning distilbert-base-multilingual-cased for typo detection using NER

In [0]:
import json


with open('/content/github-typo-corpus.v1.0.0.jsonl', 'r') as f:
  sample_file = f.readlines()
  for sample in sample_file:
    obj = json.loads(sample)
    if(len(obj['edits'][0]['tgt']['text'].split()) == 1):
      with open('train.txt', 'a') as f2:
         print(obj['edits'][0]['src']['text'].strip() + " " + "typo", file=f2)
         print(obj['edits'][0]['tgt']['text'].strip() + " " + "ok", file=f2)

In [0]:
!tail -50 train.txt

In [31]:
!wc -l train.txt

19244 train.txt


In [0]:
!cat train.txt | tail -2000 > dev.txt
!cat train.txt | head -17244 > train2.txt
!rm train.txt
!mv train2.txt train.txt

In [0]:
!rm ner_dataset/*

In [34]:
!python3 preprocess.py train.txt "distilbert-base-multilingual-cased" 128 > /content/ner_dataset/train.txt
!python3 preprocess.py dev.txt "distilbert-base-multilingual-cased" 128 > /content/ner_dataset/dev.txt

2020-04-25 14:25:08.966446: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 466/466 [00:00<00:00, 327kB/s]
Downloading: 100% 996k/996k [00:00<00:00, 2.66MB/s]
2020-04-25 14:25:16.128161: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [36]:
!python /content/transformers/examples/ner/run_ner.py --data_dir /content/ner_dataset \
--model_type distilbert \
--labels /content/labels.txt \
--model_name_or_path distilbert-base-multilingual-cased \
--output_dir model_output \
--max_seq_length 128 \
--num_train_epochs 30 \
--per_gpu_train_batch_size 128 \
--overwrite_output_dir \
--save_steps 1000 \
--do_train \
--do_eval

2020-04-25 14:26:31.243378: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/25/2020 14:26:32 - INFO - transformers.training_args -   PyTorch: setting up devices
04/25/2020 14:26:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/25/2020 14:26:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='model_output', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=128, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=30.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1)
04/25/20

In [0]:
!mv model_output distilbert-base-multi-cased-finetuned-typo-detection

In [38]:
!transformers-cli upload ./distilbert-base-multi-cased-finetuned-typo-detection

2020-04-25 14:34:24.151840: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
About to upload file /content/distilbert-base-multi-cased-finetuned-typo-detection/vocab.txt to S3 under filename distilbert-base-multi-cased-finetuned-typo-detection/vocab.txt and namespace mrm8488
About to upload file /content/distilbert-base-multi-cased-finetuned-typo-detection/special_tokens_map.json to S3 under filename distilbert-base-multi-cased-finetuned-typo-detection/special_tokens_map.json and namespace mrm8488
About to upload file /content/distilbert-base-multi-cased-finetuned-typo-detection/tokenizer_config.json to S3 under filename distilbert-base-multi-cased-finetuned-typo-detection/tokenizer_config.json and namespace mrm8488
About to upload file /content/distilbert-base-multi-cased-finetuned-typo-detection/config.json to S3 under filename distilbert-base-multi-cased-finetuned-typo-detection/config.json and namespace mrm8488
Ab